### Recommendations with MovieTweetings: Most Popular Recommendation

Now that you have created the necessary columns we will be using throughout the rest of the lesson on creating recommendations, let's get started with the first of our recommendations.

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t
from sqlalchemy import create_engine

%matplotlib inline

# Read in the datasets
# movies = pd.read_csv('movies_clean.csv')
# reviews = pd.read_csv('reviews_clean.csv')
# del movies['Unnamed: 0']
# del reviews['Unnamed: 0']

In [2]:
engine = create_engine('sqlite:///'+ '../data/TwitterMovies.db')

In [34]:
movie_ranking = pd.read_sql_table('movie_ranking',engine)

In [35]:
movie_ranking.head()

,movie,genre,date,1800s,1900s,2000s,Horror,Biography,Comedy,Adventure,...,Film-Noir,Talk-Show,nan,Short,Animation,Sci-Fi,Fantasy,avg_rating,num_ratings,last_rating
0,MSG 2 the Messenger (2015),Comedy|Drama|Fantasy|Horror,2015,0,0,1,1,0,1,0,...,0,0,0,0,0,0,1,10.0,48,2016-08-15 01:16:50
1,Avengers: Age of Ultron Parody (2015),Short|Comedy,2015,0,0,1,0,0,1,0,...,0,0,0,1,0,0,0,10.0,28,2016-01-08 08:44:43
2,Five Minutes (2017),Short|Comedy,2017,0,0,1,0,0,1,0,...,0,0,0,1,0,0,0,10.0,22,2019-04-21 06:29:19
3,Selam (2013),Drama|Romance,2013,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,10.0,10,2015-05-11 06:56:01
4,Let There Be Light (2017),Drama,2017,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,10.0,7,2019-12-26 00:27:47


In [3]:
movies = pd.read_sql_table('movies',engine)

In [4]:
movies.head()

,movie_id,movie,genre,date,1800s,1900s,2000s,Horror,Biography,Comedy,...,War,Music,Family,Film-Noir,Talk-Show,nan,Short,Animation,Sci-Fi,Fantasy
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1894,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,0000012,The Arrival of a Train (1896),Documentary|Short,1896,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,25,The Oxford and Cambridge University Boat Race ...,None,1895,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0000091,Le manoir du diable (1896),Short|Horror,1896,1,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0


In [5]:
reviews = pd.read_sql_table('reviews',engine)

In [6]:
reviews.head()

,user_id,movie_id,rating,timestamp,date
0,1,0114508,8,1381006850,2013-10-06 05:00:50
1,2,0358273,9,1579057827,2020-01-15 11:10:27
2,2,10039344,5,1578603053,2020-01-10 04:50:53
3,2,6751668,9,1578955697,2020-01-14 06:48:17
4,2,7131622,8,1579559244,2020-01-21 06:27:24


#### 1. How To Find The Most Popular Movies

For this notebook, we have a single task.  The task is that no matter the user, we need to provide a list of the recommendations based on simply the most popular items.

For this task, we will consider what is "most popular" based on the following criteria:

* A movie with the highest average rating is considered best
* With ties, movies that have more ratings are better
* A movie must have a minimum of 5 ratings to be considered among the best movies
* If movies are tied in their average rating and number of ratings, the ranking is determined by the movie that is the most recent rating

With these criteria, the goal for this notebook is to take a **user_id** and provide back the **n_top** recommendations.  Use the function below as the scaffolding that will be used for all the future recommendations as well.

In [9]:
movie_ratings = reviews.groupby('movie_id')['rating']

In [11]:
avg_ratings = movie_ratings.mean()

In [12]:
num_ratings = movie_ratings.count()

In [19]:
last_rating = reviews.groupby('movie_id').max()['date']

In [22]:
type(num_ratings)

pandas.core.series.Series

In [23]:
rating_count_df = pd.DataFrame({'avg_rating': avg_ratings, 'num_ratings': num_ratings, 'last_rating': last_rating})

In [24]:
rating_count_df.head()

,avg_rating,num_ratings,last_rating
movie_id,,,
000000000106519,9.0,1,2018-09-22 02:55:34
0000008,5.0,1,2014-04-09 02:20:11
0000010,10.0,1,2014-10-10 02:15:53
0000012,10.0,1,2015-08-11 07:16:19
0000091,6.0,3,2019-07-12 18:48:46


In [25]:
movie_recs = movies.set_index('movie_id').join(rating_count_df)

In [26]:
ranked_movies = movie_recs.sort_values(['avg_rating', 'num_ratings', 'last_rating'], ascending=False)

In [27]:
ranked_movies = ranked_movies[ranked_movies['num_ratings'] > 4]

In [29]:
ranked_movies

,movie,genre,date,1800s,1900s,2000s,Horror,Biography,Comedy,Adventure,...,Film-Noir,Talk-Show,nan,Short,Animation,Sci-Fi,Fantasy,avg_rating,num_ratings,last_rating
movie_id,,,,,,,,,,,,,,,,,,,,,
4921860,MSG 2 the Messenger (2015),Comedy|Drama|Fantasy|Horror,2015,0,0,1,1,0,1,0,...,0,0,0,0,0,0,1,10.000000,48,2016-08-15 01:16:50
5262972,Avengers: Age of Ultron Parody (2015),Short|Comedy,2015,0,0,1,0,0,1,0,...,0,0,0,1,0,0,0,10.000000,28,2016-01-08 08:44:43
6662050,Five Minutes (2017),Short|Comedy,2017,0,0,1,0,0,1,0,...,0,0,0,1,0,0,0,10.000000,22,2019-04-21 06:29:19
2737018,Selam (2013),Drama|Romance,2013,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,10.000000,10,2015-05-11 06:56:01
5804314,Let There Be Light (2017),Drama,2017,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,10.000000,7,2019-12-26 00:27:47
2560840,"Quiet Riot: Well Now You're Here, There's No W...",Documentary|Music,2014,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,10.000000,6,2016-01-23 08:30:44
2219210,Crawl Bitch Crawl (2012),Horror|Sci-Fi|Thriller,2012,0,0,1,1,0,0,0,...,0,0,0,0,0,1,0,10.000000,6,2013-07-23 07:30:52
9882084,Chasing Happiness (2019),Documentary,2019,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,10.000000,5,2019-06-29 11:37:46
4448444,Make Like a Dog (2015),Short|Comedy|Drama,2015,0,0,1,0,0,1,0,...,0,0,0,1,0,0,0,10.000000,5,2017-09-09 21:51:48


In [63]:
def load_data(database_filepath):
    engine = create_engine('sqlite:///' + database_filepath)
    ranked_movies = pd.read_sql_table('movie_ranking', engine)
    return ranked_movies


def popular_recommendations(user_id, n_top, ranked_movies):
    '''
    INPUT:
    user_id - the user_id of the individual you are making recommendations for
    n_top - an integer of the number recommendations you want back
    OUTPUT:
    top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''
    # Do stuff
    top_movies = list(ranked_movies['movie'][:n_top])

    return top_movies  # a list of the n_top movies as recommended


def popular_recs_filtered(user_id, n_top, ranked_movies, years=None, genres=None):
    '''
    REDO THIS DOC STRING

    INPUT:
    user_id - the user_id (str) of the individual you are making recommendations for
    n_top - an integer of the number recommendations you want back
    ranked_movies - a pandas dataframe of the already ranked movies based on avg rating, count, and time
    years - a list of strings with years of movies
    genres - a list of strings with genres of movies

    OUTPUT:
    top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''
    # Filter movies based on year and genre
    if years is not None:
        ranked_movies = ranked_movies[ranked_movies['date'].isin(years)]

    if genres is not None:
        num_genre_match = ranked_movies[genres].sum(axis=1)
        ranked_movies = ranked_movies.loc[num_genre_match > 0, :]

    # create top movies list
    top_movies = list(ranked_movies['movie'][:n_top])

    return top_movies


def main():
    print('Loading data...')
    ranked_movies = load_data('../data/TwitterMovies.db')

    print('Enter your name:')
    name = input()
    print('Welcome to Movie Recommendation from Leo, ' + name)
    print('Any specific production year for the movies?')
    years = []
    years_input = input()
    years.append(years_input)
    print('Noted. Any particular genre?')
    genres = []
    genres_input = input()
    genres.append(genres_input)
    print('Last question, how many recommendations?')
    n_top = int(input())

    top_movies = popular_recs_filtered('11', n_top, ranked_movies, years, genres)

    print('Thank you for your input! The list of the recommended movies are: ')
    print(top_movies)

In [65]:
main()

Loading data...
Enter your name:
Dan
Welcome to Movie Recommendation from Leo, Dan
Any specific production year for the movies?
2003
Noted. Any particular genre?
History
Last question, how many recommendations?
3
Thank you for your input! The list of the recommended movies are: /n
['The Fog of War: Eleven Lessons from the Life of Robert S. McNamara (2003)', 'Seabiscuit (2003)', 'The Corporation (2003)']


In [46]:
ranked_movies = load_data('../data/TwitterMovies.db')

In [57]:
years = input()

2003


In [58]:
years = int(years)

In [59]:
years = list(years)

TypeError: 'int' object is not iterable

In [48]:
genres = input()

History


In [60]:
list(genres)

['H', 'i', 's', 't', 'o', 'r', 'y']

In [44]:
popular_recs_filtered('70000', 100, ranked_movies, years = [2003], genres=['History', 'News'])

['The Fog of War: Eleven Lessons from the Life of Robert S. McNamara (2003)',
 'Seabiscuit (2003)',
 'The Corporation (2003)',
 'Cold Mountain (2003)',
 'Master and Commander: The Far Side of the World (2003)',
 'Shattered Glass (2003)',
 'Ned Kelly (2003)']

In [30]:
def popular_recommendations(user_id, n_top, ranked_movies):
    '''
    INPUT:
    user_id - the user_id of the individual you are making recommendations for
    n_top - an integer of the number recommendations you want back
    OUTPUT:
    top_movies - a list of the n_top recommended movies by movie title in order best to worst
    '''
    # Do stuff
    top_movies = list(ranked_movies['movie'][:n_top])
    
    return top_movies # a list of the n_top movies as recommended

In [31]:
popular_recommendations(11, 7, ranked_movies)

['MSG 2 the Messenger (2015)',
 'Avengers: Age of Ultron Parody (2015)',
 'Five Minutes (2017)',
 'Selam (2013)',
 'Let There Be Light (2017)',
 "Quiet Riot: Well Now You're Here, There's No Way Back (2014)",
 'Crawl Bitch Crawl (2012)']

In [33]:
t.popular_recommendations('1', 20, ranked_movies)

AttributeError: module 'tests' has no attribute 'popular_recommendations'

Usint the three criteria above, you should be able to put together the above function.  If you feel confident in your solution, check the results of your function against our solution. On the next page, you can see a walkthrough and you can of course get the solution by looking at the solution notebook available in this workspace.  

In [ ]:
# Put your solutions for each of the cases here

# Top 20 movies recommended for id 1

recs_20_for_1 = # Your solution list here

# Top 5 movies recommended for id 53968
recs_5_for_53968 = # Your solution list here

# Top 100 movies recommended for id 70000
recs_100_for_70000 = # Your solution list here

# Top 35 movies recommended for id 43
recs_35_for_43 = # Your solution list here



In [ ]:
### You Should Not Need To Modify Anything In This Cell
ranked_movies = t.create_ranked_df(movies, reviews) # only run this once - it is not fast

# check 1 
assert t.popular_recommendations('1', 20, ranked_movies) == recs_20_for_1,  "The first check failed..."
# check 2
assert t.popular_recommendations('53968', 5, ranked_movies) == recs_5_for_53968,  "The second check failed..."
# check 3
assert t.popular_recommendations('70000', 100, ranked_movies) == recs_100_for_70000,  "The third check failed..."
# check 4
assert t.popular_recommendations('43', 35, ranked_movies) == recs_35_for_43,  "The fourth check failed..."

print("If you got here, looks like you are good to go!  Nice job!")

**Notice:** This wasn't the only way we could have determined the "top rated" movies.  You can imagine that in keeping track of trending news or trending social events, you would likely want to create a time window from the current time, and then pull the articles in the most recent time frame.  There are always going to be some subjective decisions to be made.  

If you find that no one is paying any attention to your most popular recommendations, then it might be time to find a new way to recommend, which is what the next parts of the lesson should prepare us to do!


### Part II: Adding Filters

Now that you have created a function to give back the **n_top** movies, let's make it a bit more robust.  Add arguments that will act as filters for the movie **year** and **genre**.  

Use the cells below to adjust your existing function to allow for **year** and **genre** arguments as **lists** of **strings**.  Then your ending results are filtered to only movies within the lists of provided years and genres (as `or` conditions).  If no list is provided, there should be no filter applied.

You can adjust other necessary inputs as necessary to retrieve the final results you are looking for!

Try writing a few tests against the test function in our test function.  Below returns the top 20 movies for user 1 based on the specified year and genre filters.  Does yours return the same? 

```
t.popular_recs_filtered('1', 20, ranked_movies, years=['2015', '2016', '2017', '2018'], genres=['History'])
```